# Investigate normal expression of cis proteins that show equivalence between samples with and without the event

We want to know why these proteins don't show a significant change even when the arm is deleted. It's possible that they have low expression to begin with, so the change from the deletion isn't noticeable.

Unfortunately, we can only get relative expression levels from the CPTAC data, not absolute expression levels. So instead of using CPTAC data, we're going to be working with tissue-specific absolute expression data from this paper: Wang D, Eraslan B, Wieland T, et al. A deep proteome and transcriptome abundance atlas of 29 healthy human 
tissues. Mol Syst Biol. 2019;15(2):e8503. Published 2019 Feb 18. doi:10.15252/msb.20188503

We downloaded the specific table, Table 5, from https://www.embopress.org/action/downloadSupplement?doi=10.15252%2Fmsb.20188503&file=msb188503-sup-0007-TableEV5.zip

In [1]:
import pandas as pd
import numpy as np
import os
import cnvutils
import scipy.stats
import altair as alt
from toolz import pipe

ModuleNotFoundError: No module named 'gprofiler'

In [ ]:
alt.data_transformers.disable_max_rows()

def json_dir(data, data_dir):
    os.makedirs(data_dir, exist_ok=True)
    return pipe(data, alt.to_json(filename=os.path.join(data_dir, "{prefix}-{hash}.{extension}")) )

alt.data_transformers.register("json_dir", json_dir)
alt.data_transformers.enable("json_dir", data_dir="plot_data")

In [ ]:
# These variables specify which chromosome and arm we're working on, and whether to do cis or trans effects
params = cnvutils.load_params(os.path.join("..", "data", "params.json"))
CHROMOSOME = params["CHROMOSOME"]
ARM = params["ARM"]
CIS_OR_TRANS = "cis"

## Transform to log(x + 1), and exclude zeros

Based on the plots in the `00_setup/00_normal_dist_exploration` notebook, it looks like our best option is to use a log(x + 1) scale, and exclude zeros. From a biological perspective, I'm fine with excluding the zeros, because proteins that aren't expressed seem a different class from proteins that just have low expression. Although it is important to remember that it's also possible that proteins with zero copies may have just been too low in expression to be detected. Nevertheless, the fact that there's such a huge number of proteins with zero copies suggests that they aren't just the few escaping detection.

In [ ]:
expr = cnvutils.get_normal_expr_table()

expr_long = expr.\
drop(
    columns=["Gene_ID", "Protein_ID"]
).\
melt(
    id_vars="Gene_name",
    var_name="tissue_type",
    value_name="prot_copy_count"
)

expr_plus1_log10 = expr_long.assign(prot_copy_count=np.log10(expr_long["prot_copy_count"] + 1))
expr_transf = expr_plus1_log10[expr_plus1_log10["prot_copy_count"] > 0]

## Calculate "low" cutoff for each tissue type

In [ ]:
low_cutoffs = expr_transf.groupby("tissue_type").quantile(0.25)
low_cutoffs

## Compare expression of equivalent genes

In [ ]:
equiv_prots = pd.read_csv(
    os.path.join("..", "data", f"{CHROMOSOME}{ARM}_{CIS_OR_TRANS}_common_equiv.tsv"), 
    sep="\t"
)
equiv_prots

In [ ]:
commonly_equiv = equiv_prots[equiv_prots["num_cancers"] >= 3]
sel_expr = expr.loc[
    expr["Gene_name"].isin(commonly_equiv["protein"]),
    ["Gene_name", "Protein_ID", "Colon", "Esophagus", "Lung", "Ovary"]
].\
set_index(["Gene_name", "Protein_ID"]).\
transpose()

sel_expr.columns = sel_expr.columns.to_flat_index()

# Transform the data to log10(x + 1), join in cutoffs column
sel_expr = sel_expr.\
apply(lambda x: np.log10(x + 1), axis="columns").\
join(low_cutoffs, how="left")

sel_expr = sel_expr.apply(lambda x: x <= sel_expr["prot_copy_count"]).\
drop(columns="prot_copy_count").\
transpose()

sel_expr.index = pd.MultiIndex.from_tuples(sel_expr.index).droplevel(1)

sel_expr

In [ ]:
sel_expr.shape

In [ ]:
sel_expr.sum(axis="index")

### Plot on the distribution

In [ ]:
def plot_cancer_type_normal_dist(cancer_type, tissue_type, expr_data, sel_genes):
    
    # Select the expression data for our chosen tissue type
    expr_data = expr_data[expr_data["tissue_type"] == tissue_type]
    
    dist = alt.Chart().mark_bar().encode(
        x=alt.X(
            "prot_copy_count",
            bin=alt.Bin(step=0.25),
            title="Protein copy counts (log transformed)"
        ),
        y=alt.Y(
            "count()",
            title="Number of proteins"
        )
    ).properties(
        width=800
    )

    # Add vertical lines at the expression levels of the proteins that didn't show a cis effect
    lines_no_cis = [
        alt.Chart().mark_rule(color=alt.Value("#d10000")).encode(
            x=f"{gene}:Q"
        )

        for gene in sel_genes
    ]

    # Get the expression levels for those proteins
    expr_no_cis = expr_data.\
    loc[expr_data["Gene_name"].isin(sel_genes)].\
    set_index("Gene_name")["prot_copy_count"].\
    astype(str).\
    to_dict()

    # Layer all the charts together
    chart = alt.layer(
        dist,
        *lines_no_cis,
        data=expr_data
    ).transform_calculate(
        **expr_no_cis
    ).properties(
        title=cancer_type
    )
    
    return chart

In [ ]:
cancer_tissue = {
    "colon": "Colon",
    "hnscc": "Esophagus",
    "luad": "Lung",
    "lscc": "Lung",
    "ovarian": "Ovary"
}

alt.vconcat(*[
    plot_cancer_type_normal_dist(
        cancer_type=cancer, 
        tissue_type=tissue,
        expr_data=expr_transf,
        sel_genes=sel_expr.index.tolist()
    )
    
    for cancer, tissue in cancer_tissue.items()
]).properties(
    title=f"Normal expression of proteins that showed no cis effect for {CHROMOSOME}{ARM}"
).configure_title(
    anchor="middle",
    fontSize=14
)

## Look at cancers individually

In [ ]:
each_cancer_equiv = pd.\
read_csv(os.path.join("..", "data", f"{CHROMOSOME}{ARM}_{CIS_OR_TRANS}_equiv.tsv"), sep="\t")

In [ ]:
cancer_tissue = {
    "colon": "Colon",
    "hnscc": "Esophagus",
    "luad": "Lung",
    "lscc": "Lung",
    "ovarian": "Ovary"
}

cancer_expr = {}

for cancer_type, tissue_type in cancer_tissue.items():
    
    cancer_equiv = each_cancer_equiv[each_cancer_equiv["cancer_type"] == cancer_type]
    sel_expr = expr.loc[
        expr["Gene_name"].isin(cancer_equiv["protein"]),
        ["Gene_name", "Protein_ID", tissue_type]
    ].\
    set_index(["Gene_name", "Protein_ID"]).\
    transpose()

    sel_expr.columns = sel_expr.columns.to_flat_index()

    # Transform the data to log10(x + 1), join in cutoffs column
    sel_expr = sel_expr.\
    apply(lambda x: np.log10(x + 1), axis="columns").\
    join(low_cutoffs, how="left")

    sel_expr = sel_expr.apply(lambda x: x <= sel_expr["prot_copy_count"]).\
    drop(columns="prot_copy_count").\
    transpose()

    sel_expr.index = pd.MultiIndex.from_tuples(sel_expr.index).droplevel(1)
    
    print(f"{cancer_type}: {sel_expr.sum()[0]}/{sel_expr[tissue_type].size} ({sel_expr.sum()[0] / sel_expr[tissue_type].size:.2f}) below cutoff")

    cancer_expr[cancer_type] = sel_expr

In [ ]:
cancer_expr["colon"]

In [ ]:
cancer_expr["hnscc"]

In [ ]:
cancer_expr["lscc"]

In [ ]:
cancer_expr["luad"]

In [ ]:
cancer_expr["ovarian"]